In [36]:
# Library imports
import pyforest
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from pprint import pprint
from time import sleep
import time
import seaborn as sns
from sklearn.metrics import roc_auc_score

from turtle import forward
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.utils.data import DataLoader, TensorDataset

#### **Load dataset**

In [2]:
dataset = 'movielens'

train_edges = np.load('data/' + dataset + '/train_edges.npy')
test_edges = np.load('data/' + dataset + '/test_edges.npy')

user_list_train = train_edges[:, 0]
user_list_test = test_edges[:, 0]
item_list_train = train_edges[:, 1]
item_list_test = test_edges[:, 1]
rating_list_train = train_edges[:, 2].astype('float32')
rating_list_test = test_edges[:, 2].astype('float32')

n_users = max(user_list_train.max(), user_list_test.max()) + 1 
n_items = max(item_list_train.max(), item_list_test.max()) + 1
n_samples_train = len(rating_list_train)
n_samples_test = len(rating_list_test)

n_users, n_items, n_samples_train, n_samples_test

(943, 1682, 159619, 40381)

#### **Defining collaborative filtering**

In [12]:
class CollaborativeFiltering(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(CollaborativeFiltering, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        drop_u = nn.Dropout(p = 0.3)
        drop_i = nn.Dropout(p = 0.3)
        u = self.user_emb(user)
        i = self.item_emb(item)
        u = drop_u(u)
        i = drop_i(i)
        dot = (u * i).sum(dim = 1)
        return torch.sigmoid(dot)

class CF1(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(CF1, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)
        self.fc = nn.Linear(n_factors * 3, 1)

    def forward(self, user, item):
        drop_u = nn.Dropout(p = 0.3)
        drop_i = nn.Dropout(p = 0.3)
        u = self.user_emb(user)
        i = self.item_emb(item)
        u = drop_u(u)
        i = drop_i(i)
        dot = (u * i)
        x = torch.concat([u, i, dot], dim = 1)
        x = self.fc(x)
        return torch.sigmoid(x)

class NCF(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(NCF, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)
        self.fc1 = nn.Linear(n_factors * 2, n_factors)
        self.fc2 = nn.Linear(n_factors, 1)

    def forward(self, user, item):
        tanh = nn.Tanh()
        sigmoid = nn.Sigmoid()
        swish = nn.SiLU()

        u = self.user_emb(user)
        i = self.item_emb(item)
        x = torch.concat([u, i], dim = 1)
        x = swish(x)
        x = self.fc1(x)
        x = swish(x)
        x = self.fc2(x)
        x = sigmoid(x)
        return x

def get_accuracy(y_hat, y):
    y = y.clone().int()
    y_hat = (y_hat.clone() > 0.5).int()
    accuracy = (y == y_hat).sum() / len(y)
    return accuracy.item()

#### **Test collaborative filtering on unseen data**

In [17]:
use_gpu = 1
if use_gpu == -1:
    device = 'cpu'
else:
    device = torch.device('cuda:{}'.format(str(use_gpu)) if torch.cuda.is_available() else 'cpu')

users = torch.tensor(user_list_train, device = device)
items = torch.tensor(item_list_train, device = device)
ratings = torch.tensor(rating_list_train, device = device, requires_grad = True)

users_test = torch.tensor(user_list_test, device = device)
items_test = torch.tensor(item_list_test, device = device)
ratings_test = torch.tensor(rating_list_test, device = device)

# ratings = ratings.reshape((n_samples_train, 1))
# ratings_test = ratings_test.reshape((n_samples_test, 1))

In [18]:
# lr = 1 and T = 1000, Adam giving 77% accuracy on test data
# lr = 1.1 and T = 500, Adam giving 77.2% accuracy on test data
# lr = 1.4 and T = 250, Adam giving 76.8% accuracy on test data
# lr = 1.8 and T = 250, Adam giving 77.2% accuracy on test data
# lr = 1.6 and T = 250, Adam giving 77% accuracy on test data but making meta-gradients infinite
# lr = 500, T = 300, SGD, Dropout = 0.3 gives 75.58% accuracy on test data
# lr = 1000, T = 300, SGD, Dropout = 0.3, seed = 0 gives 76% accuracy on test data
# lr = 1000, T = 300, SGD, Dropout = 0.3, seed = 50 gives 76.18% accuracy on test data

n_factors = 64
T = 300
seed = 50

# for lr in list(range(1, 500, 5)):
for lr in [1000]:

    model = CollaborativeFiltering(n_users, n_items, n_factors)
    model.to(device)

    torch.manual_seed(seed)
    for layer in model.children():
        layer.reset_parameters()
    # optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)
    loss_fn = torch.nn.BCELoss(reduction = 'mean')

    model.train()
    for _ in range(T):
        y_hat = model(users, items)
        loss = loss_fn(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward(retain_graph = True)
        optimizer.step()

    model.eval()
    y_hat = model(users, items)
    print('lr: ', lr)
    print('Training accuracy: ', get_accuracy(y_hat, ratings))
    loss = loss_fn(y_hat, ratings)
    meta_grad = torch.autograd.grad(loss, ratings)[0]
    print('max meta grad: ', meta_grad.max())
    print('min meta grad: ', meta_grad.min())
    print(meta_grad)
    y_hat = model(users_test, items_test)
    print('Testing accuracy: ', get_accuracy(y_hat, ratings_test))
    print()


lr:  1000
Training accuracy:  0.9091523885726929
max meta grad:  tensor(0.0003, device='cuda:1')
min meta grad:  tensor(-inf, device='cuda:1')
tensor([ 3.1337e-06, -1.8015e-05, -4.1825e-05,  ..., -2.6589e-06,
         5.7238e-05,  1.4330e-06], device='cuda:1')
Testing accuracy:  0.7618434429168701



#### **Exploring AUC metric for evaluating CF model**

In [35]:
# So far, using AUROC instead of accuracy seems promising

In [33]:
y_hat = model(users_test, items_test)
y_pred = y_hat.detach().clone().to('cpu').numpy()
y = ratings_test.detach().clone().to('cpu').numpy()
print(y_pred)
print(y)
roc_auc_score(y, y_pred)

[0.8015251  0.9998387  0.3214179  ... 0.7845767  0.98115957 0.9961748 ]
[1. 1. 1. ... 1. 1. 1.]


0.8359565661645652

In [34]:
y_hat = model(users, items)
y_pred = y_hat.detach().clone().to('cpu').numpy()
y = ratings.detach().clone().to('cpu').numpy()
print(y_pred)
print(y)
roc_auc_score(y, y_pred)

[7.7284789e-01 2.8575003e-01 9.9942243e-01 ... 9.7379261e-01 7.6284632e-05
 5.8251357e-01]
[0. 1. 1. ... 1. 0. 0.]


0.9722345113132543